# Medidas de Centralidad y Análisis de Propagación de Enfermedades Animales

<!-- NOTA: Este titulo está muy largo, buscar uno más cortico -->

Introducción a la Optimización — 2022-I\
Universidad Nacional de Colombia

<!-- NOTA: Escribir correos -->
[Sergio Alejandro Vargas](mailto:savargasqu@unal.edu.co)\
Santiago Jiménez \
Daniel Felipe Quiñones

## Introducción

<!-- Problema -->
La difusión de enfermedades animales es una amenaza latente a la seguridad alimentaria en Colombia.
Por esto, estamos interesados en predecir cuales son los municipios con mayor probabilidad de ser centros de propagación de enfermedades.
Esta información puede ser usada a futuro para ejecutar acciones sanitarias que limiten la propagación de enfermedades animales.

<!-- ¿qué? -->
Podemos estudiar la red de transporte de ganado en Colombia como un grafo,
cuyos nodos son municipios y cuyos arcos son rutas de transporte.
A partir de este grafo, definimos una medida de centralidad que nos permita estimar cuales son los municipios en los se puede dar una mayor propagación de enfermedades por su posición en la red de transporte, relativo a los otros municipios y las rutas entre ellos.

<!--
Para nuestro caso miraremos el ganado porcino y basado en el numero de individuos concentrados en diferentes municipios haremos un rankeo para ver cuales son los de mayor probabilidad de infeccion.

NOTA: Esto no está bien. La clasificación no se hace sobre la cantidad de animales sino sobre la topologia del grafo.
-->

<!-- ¿como? -->
Primero, preparamos datos recolectados por la Asociación de Porcicultores de Colombia (Porkcolombia); con estos datos construimos un grafo que representa la red de transporte de ganado porcino en el país; luego, calculamos diferentes medidas de centralidad sobre el grafo; por último, planteamos una nueva medida de centralidad construida a partir de la combinación convexa de la medidas ya conocidas, y la calculamos solucionando un problema de optimización.Esta nueva medida nos va a permitir, no solo sobreestimar <!-- ¿no sería _no sobreestimar_ --> alguna caracteristica del grafo sobre su topologia, sino tambien encontrar una medida mas optima entre las diferentes posibilidades.[1]

In [1]:
# Trabajaremos con los siguientes paquetes:

from operator import concat
import cvxpy as cp
import networkx as nx
import numpy as np
import pandas as pd

## Preparación de datos

Actualmente, Los datos provistos por Porkcolombia no son públicos. Las siguientes funciones se evaluan sobre los datos originales y se preservan en el notebook por razones de reproducibilidad.

Para leer y organizar los datos usamos un DataFrame de `pandas`.



<!-- las fechas, el tipo y número de animales también pueden ser relevantes para un análisis más detallado. -->

<!-- Edit comentario: preservaria las fechas para hacer analisis por ciertos periodos de tiempo. El tipo de animales por ahora no creo que sea muy relevante. Daniel -->


Primero, vamos a tomar cada municipio (y no cada predio) como un solo nodo, por lo que no nos interesan las rutas que van de un predio a otro dentro del mismo municipio, ya que formarían bucles en el grafo. Eliminamos las entradas del dataframe en las que el origen y el destino son iguales.

In [2]:
def remove_loops(df):
    return df.loc[df['MUNICIPIO_ORIGEN'] != df['MUNICIPIO_DESTINO']]

Como los datos no son públicos, reemplazamos el nombre de cada municipio con una etiqueta numérica. Los arcos (rutas) no se modifican, luego el grafo obtenido es equivalente.

In [3]:
"Return new dataframe with anonymous labels for origin and destination"
def sanitize_labels(df):
    # Some municipalities have the same name, so we also need to consider the department.
    map_concat = lambda xs, ys: list(map(concat, xs, ys))

    # Get the list of all origins and destinations.
    origins = map_concat(df['DEPARTAMENTO_ORIGEN'], df['MUNICIPIO_ORIGEN'])
    destinations = map_concat(df['DEPARTAMENTO_DESTINO'], df['MUNICIPIO_DESTINO'])

    # Get sorted list of all unique places.
    places = sorted(set(origins).union(set(destinations)))

    # Label each name with its position in the sorted list  
    label = lambda names: list(map(places.index, names))

    return pd.DataFrame({
        'origin': label(origins),
        'destination': label(destinations),
         # 'total': df['TOTAL_ANIMALES'], # only needed for weighted graphs (needs to be aggregated over a time interval)
    })

Solo tenemos que considerar cada ruta una vez para construir el grafo.

In [4]:
def without_duplicates(df):
    df.drop_duplicates(['origin', 'destination'], ignore_index=True, inplace=True)
    return

Guardamos los datos etiquetados en un nuevo archivo. Este es el archivo de datos que se encuentra en el repositorio.

In [5]:
def save_clean_data(input, output):
    df = pd.read_csv(input)
    df = remove_loops(df)
    df = sanitize_labels(df)
    without_duplicates(df)
    df.to_csv(output, index=False)
    return

In [7]:
# This cell only needs to run on the original dataset.
PATH = './'
save_clean_data(PATH + 'data.csv', PATH + 'clean_data.csv')

Ahora, para construir el grafo es suficiente formar un arco por cada fila del dataset.

In [8]:
_data = pd.read_csv(PATH + 'clean_data.csv')
G = nx.from_pandas_edgelist(_data, source='origin', target='destination')

## Medidas de Centralidad

Intuitivamente, una medida de centralidad determina la importancia de un nodo dentro de un grafo. Aplicado a nuestro problema, buscamos una medida de centralidad que indique cuales son los municipios más importantes en la red de transporte de ganado. Esta importancia se puede cuantificar de varias maneras,
pero en general, podremos suponer que si se presenta un brote de enfermedad en un municipio $m$, cortar las rutas de $m$ hacía los municipios importantes cercanos provería la mayor seguridad sanitaria.

Formalmente, definiremos una medida de centralidad de la siguiente manera: Sea $G = (V, E)$ un grafo no-dirigido con vertices $V$ y arcos $E$. Una medida de centralidad es una función de valor real $c: V \to ℝ$. Como existe un orden total para los reales, siempre podemos comparar la centralidad de dos vertices en un grafo.[2]

Como las medidas de centralidad se calculan a partir de diferentes características de un nodo, la clasificación de un nodo no es la misma para todos las medidas de centralidad. Para dar una mejor estimación de la importancia de un nodo, proponemos usar una nueva medida de centralidad construida a partir de la combinación convexa de medidas de centralidad ya existentes.

Ahora, para construir una nueva medida de centralidad, tenemos que poder operar las centralidades ya existentes sobre un mismo dominio, ya que en general, no son comparables y pueden variar drasticamente con el tamaño de un grafo. Para esto, normalizamos cada medida para ajustarla al intervalo $[0, 1]$ donde 1 es la mayor centralidad y 0 es la menor.


Antes de definir las medidas de centralidad, definimos la _distancia geodésica_ de dos nodos como el número de vertices en el camino más corto entre los dos nodos. Si no hay un camino, la distancia es infinita.

Las medidas de centralidad con las que trabajamermos son las siguientes:

<!-- definición y normalización -->

* Centralidad de grado (_degree centrality_): Esta medida de centralidad le asigna a cada nodo el número de vértices que inciden sobre él (el grado del nodo).[2]

<!--
 Esta medida de centralidad, si bien es muy sencilla, en ocasiones puede dar mejores resultados que medidas más sofisticadas. Una desventaja obvia es que esta medida no tiene en cuenta la "calidad" de las conexiones entre nodos, así, dos nodos que tengan igual número de conexiones, pueden parecer igualmente relevantes bajo esta medida, sin serlo necesariamente.

Esto sobra. Se puede poner en las conclusiones -->

* Centralidad de cercanía (_closeness centrality_): Esta medida le asigna a cada nodo $n$ el reciproco de la sumas de la distancias del nodo $n$ a los otros nodos, es decir, si en promedio la distancia del nodo $n$ a otros nodos es corta, su centralidad es más alta.[3]
$$C_{closeness}(x) = \frac{1}{∑_{d(x,y)<∞}d(y,x)}$$
  Como la distancia entre dos nodos es infinita si estos no están conectados,
  no incluimos estas distancias en la suma.

* Centralidad de intermediación (_betweenness centrality_): Esta medida le asigna a cada nodo la frecuencia con la que el nodo aparece en la geodésica de otros dos nodos cualesquiera en el grafo, es decir si $σ_{yz}$ es el número de caminos cortos de $y$ a $z$, y $σ_{yz}(x)$ es el número de caminos que pasan por $x$, la intermedación se define como:
$$C_{centrality}(x) = ∑_{y,z \neq x, σ_{yz}\neq 0}\frac{σ_{yz}(x)}{σ_{yz}}$$
  Si muchos caminos cortos pasan por $x$, este es un punto con una alta centralidad de intermediación.[3]

* Centralidad de vector propio (_eigenvector centrality_): Un grafo puede ser representado por una matriz de adyacencia $A$, donde la entrada $a_{ij}$ es igual a 1 si el nodo $n_i$ se conecta con el nodo $n_j$ e igual a 0 de lo contrario. La matriz es simétrica si el grafo no es dirigido. Para esta matriz existe un valor propio positivo $\lambda$ cuya norma es mayor que la del resto de valores propios, y además existe un vector propio $x$ con componentes positivas asociado a $\lambda$, es decir un vector que satisface $Ax = λx$ y $x ≽ 0$, entonces el valor de la i-ésima entrada de $x$ es el valor de centralidad del i-ésimo nodo. Esta medida le asignará un valor mayor a un nodo, si este nodo está conectado con otros nodos que tienen muchas conecciones.

Calculamos cada medida de centralidad usando `networkx`. Estas funciones retornan un diccionario con entradas `node: value` ya normalizadas.

# Interpretación epidemiológica

Podemos darle a cada medida de centralidad una interpretación particular de acuerdo al problema que estamos tratando; esto es; la propogación de una cierta enfermedad a través de redes de transporte animal:

Centralidad de grado: Esta medida de centralidad nos indica en cuáles nodos es más probable que surja una enfermedad infecciosa(por recibir una mayor cantidad de animales de otros nodos)

Centrilidad de cercanía: Esta medida de centralidad nos indica desde qué nodos una supuesta enfermedad infecciosa se propagaría con mayor rapidez. Es decir, los nodos que puntúen alto en esta medida de centralidad serán aquellos desde los cuales se puede propagar una enfermedad muy infecciosa a prácticamente todos los nodos en poco tiempo

Centralidad de intermedación: Esta medida de centralidad nos indica cuáles son los nodos que jugarían un papel crucial en la propagación de una supuesta epidemia, por estar muy bien conectados con los demás nodos y porque sirven de enlace entre nodos que de otra manera estarían desconectados

Centralidad de vector propio: Esta medida de centralidad nos indica cuáles son los nodos a los que es más probable que llegue la infección después de un cierto tiempo, por estar dichos nodos conectados con nodos que juegan un papel importante en la red de transporte animal

In [9]:
def get_centralities(graph):
    return {
        'betweenness': nx.betweenness_centrality(graph),
        'closeness': nx.closeness_centrality(graph),
        'degree': nx.degree_centrality(graph),
        'eigenvector': nx.eigenvector_centrality(graph),
    }

A partir de estas medidas podemos construir una matriz $A$ donde las filas corresponden al i-ésimo nodo, y las columnas corresponden a los valores de cada medida de centralidad. Así, para la entrada $a_{ij}$ tenemos el valor del nodo $i$ para la j-ésima medida.

In [10]:
def build_matrix(graph):
    cs = get_centralities(graph)
    A = np.empty((
        len(cs['betweenness']),
        len(cs),
    ))
    i = 0
    for k in ['betweenness', 'closeness', 'degree', 'eigenvector']:
        for node in cs[k]:
            A[node, i] = cs[k][node]
        i += 1
    return A

In [11]:
A = build_matrix(G)

## Minimización de combinaciones convexas

<!--
Sean $v_1, v_2,...v_n$ los valores de centralidad de los vértices del grafo para $n$ medidas de centralidad, donde la i-ésima entrada del vector $v_j$ nos indica el valor de centralidad sobre la medida $j$ del nodo $i$.
-->

Usando la matriz de valores de centralidad $A$,
queremos encontrar un vector de valores de centralidad $w$ que considere las $n$ medidas. A cada columna de $A$ le asignamos un factor escalar $c_i$ para ajustar la contribución de la medida $i$ a $w$. Formalmente, queremos encontrar un vector $c = (c_1, c_2, …, c_n)$ que minimice

$$\min \sum_{i=1}^{n} ||w - A_i||^2$$

donde $w = \sum_{i=1}^{n} c_iA_i$ y sujeto a:

$$\sum_{i=1}^{n} c_i = 1$$
$$c_i\geq 0$$

Este es un problema de optimización convexa, puesto que la expresión a minimizar es una suma de normas, cada una de las cuales es convexa. Además la condición de que cada $c_i$ sea no negativo también respeta dicha convexidad, y por último la condición de que $\sum_i c_i = 1$ puede escribirse de la forma $1^Tc=1$, donde $c=(c_1,c_2,c_3,c_4)^T$. Claramente $1^Tc$ es una función afín. 

Por lo tanto este problema satisface las tres condiciones necesarias para que un problema de optimización sea convexo y podemos usar un _solver_ de CVXPY para resolverlo.

In [12]:
def convex_centrality(A):
    c = cp.Variable(4)
    objective = cp.Minimize(
        cp.sum_squares(A @ c - A[:, 0]) +
        cp.sum_squares(A @ c - A[:, 1]) +
        cp.sum_squares(A @ c - A[:, 2]) +
        cp.sum_squares(A @ c - A[:, 3])
    )
    constraints = [
        c >= 0,
        np.ones(4).T @ c == 1
    ]
    cp.Problem(objective, constraints).solve()
    return c.value


In [13]:
c = convex_centrality(A)

In [14]:
def get_rank(vec):
    return list(map(lambda y: y[0], sorted(enumerate(vec), key=lambda x: x[1], reverse=True)))

In [15]:
pd.DataFrame([
    get_rank(A[:, 0]),
    get_rank(A[:, 1]),
    get_rank(A[:, 2]),
    get_rank(A[:, 3]),
    get_rank(A @ c)
])

,0,1,2,3,4,5,6,7,8,9,...,904,905,906,907,908,909,910,911,912,913
0,151,100,69,101,243,650,369,8,690,730,...,853,854,855,858,860,867,869,870,874,884
1,100,151,69,101,730,8,56,46,93,54,...,145,126,581,577,546,520,165,232,0,1
2,151,100,101,69,8,243,730,650,56,93,...,802,808,812,836,853,860,869,870,874,884
3,101,100,69,8,151,730,56,93,16,46,...,561,581,546,126,577,520,165,232,0,1
4,151,100,101,69,8,730,56,243,93,46,...,145,126,581,577,546,520,165,232,0,1


## Conclusiones

1. No existe una única manera de determininar el centro de un grafo. Existen muchas medidas de centralidad diferentes y cada una de ellas nos ayuda a comprender una parte de las conexiones de los nodos de un grafo. Combinando estas medidas entre sí, obtenemos mucha más información adicional a la obtenida sólo basándondonos en una única medida de centralidad.
2. En relación a nuestro problema original de analizar la propagación de una infección animal, pudimos ver cómo cada una de las diferentes medidas de centralidad por sí sola es importante para analizar las diferentes fases de una epidemia, ya sea su posible origen, su propagación inicial, o su posible propagación final a todos los nodos de la red.
3. En los rankings obtenidos vemos que hay similaridad en los nodos y la variacion de la posicion no es muy alta. Esto debido a que cada medida resalta una caracteristica diferente del nodo, y, sin embargo, al hacer la combinacion convexa mas optima, esta preserva y encuentra el ranking con mayor relevancia general sobre la red. Asi por ejemplo vemos que los nodos 100, 101, 151, 69 estan variando en el top 3 de las diferentes medidas de centralidad, y en la solucion del problema de optimizacion esta los ubica en el top 4 del ranking (151, 100, 101, 69).
4. La teoría de optimización vista en clase juega un papel fundamental a la hora de plantear correctamente el problema de encontrar el mejor ranking que tome en cuenta todas las medidas de centralidad, pues proporciona un marco natural para estudiar las funciones a optimizar y nos permite estar seguros de la existencia de una solución óptima. Además, también se puede aplicar para resolver una variedad de problemas en teoría de grafos que impliquen optimizar una cierta variable, problemas que pueden ser planteados desde una variedad de perspectivas muy distintas a las que se trataron en este trabajo.

**Referencias**

[1]: Keng, Y., Kwa, K., McClain, C. (2021). _Convex combination of centrality measures._ The Journal of Mathematical Sociology.

[2]: Brandes, U., Erlebach T. (2005) _Network Analysis. Methodological Foundations._

[3]: Boldi, P., Vigna, S. (2013) _Axioms for centrality._ Internet Mathematics.